In [1]:
import gdn_datacollector
import datetime
import gdn_db
from googleads import adwords
import pandas as pd
import numpy as np
AUTH_FILE_PATH = '/home/tim_su/ai_optimizer/opt/ai_optimizer/googleads.yaml'
client = adwords.AdWordsClient.LoadFromStorage(AUTH_FILE_PATH)
CRITERIA_LIST = ['ADGROUP', 'URL', 'CRITERIA', 'AGE_RANGE', 'DISPLAY_KEYWORD', 'AUDIENCE']

In [117]:
DATABASE_INDEX = {
    'ADGROUP': 'adgroup_insights',
    'URL': 'url_insights',
    'CRITERIA': 'criteria_insights',
    'AUDIENCE': 'audience_insights',
    'AGE_RANGE': 'age_range_insights',
    'DISPLAY_KEYWORD': 'display_keyword_insights',
}
SCORE_INDEX = {
    'ADGROUP': 'adgroup',
    'URL': 'url',
    'CRITERIA': 'criteria',
    'AUDIENCE': 'audience',
    'AGE_RANGE': 'age_range',
    'DISPLAY_KEYWORD': 'display_keyword',
}
SCORE_COLUMN_INDEX = {
    'ADGROUP': ['campaign_id', 'adgroup_id', 'score'],
    'URL': ['campaign_id', 'url_display_name', 'score'],
    'CRITERIA': ['campaign_id', 'keyword_placement', 'keyword_id', 'score'],
    'AUDIENCE': ['campaign_id', 'audience', 'criterion_id', 'score'],
    'AGE_RANGE': ['campaign_id', 'age_range', 'criterion_id', 'score'],
    'DISPLAY_KEYWORD': ['campaign_id', 'keyword', 'keyword_id', 'score'],
}

In [149]:
class ReportField:
    # URL
    URL_LIST = [
        'ExternalCustomerId', 'CampaignId', 'AdGroupId', 'AdGroupStatus', 'Cost', 'AverageCost',
        'Impressions', 'Clicks', 'Conversions', 'AverageCpc', 'CostPerConversion', 'DisplayName', ]
    # CRITERIA,
    CRITERIA_LIST = [
        'ExternalCustomerId', 'CampaignId', 'AdGroupId', 'Criteria', 'Id', 'AdGroupStatus', 'CpmBid', 'CpcBid',
        'Cost', 'AverageCost', 'Impressions', 'Clicks', 'Conversions', 'AverageCpc', 'CostPerConversion']
    # AUDIENCE, AGE_RANGE, DISPLAY_KEYWORD
    BIDDABLE_LIST = [
        'ExternalCustomerId', 'CampaignId', 'AdGroupId', 'Criteria', 'Id', 'AdGroupStatus', 'CpmBid', 'CpcBid',
        'BiddingStrategyType', 'Cost', 'AverageCost', 'Impressions', 'Clicks', 'Conversions', 'AverageCpc', 'CostPerConversion']
    # ADGROUP
    ADGROUP_LIST = [
        'ExternalCustomerId', 'CampaignId', 'AdGroupType', 'AdGroupId', 'AdGroupStatus', 'CpmBid', 'CpvBid', 'CpcBid', 'TargetCpa',
        'BiddingStrategyType', 'Cost', 'AverageCost', 'Impressions', 'Clicks', 'Conversions', 'AverageCpc', 'CostPerConversion']

    NON_NUMERIC_LIST = [
        'Ad group type', 'Ad group state', 'Bid Strategy Type', 'Keyword', 'Age Range', 'Audience', 'Keyword / Placement', 'Criteria Display Name']

    NUMERIC_LIST = [
        'Max. CPM', 'Max. CPV', 'Max. CPC', 'Default max. CPC', 'Target CPA', 'Cost', 'Avg. Cost', 'Avg. CPC', 'Cost / conv.']

    INDEX = {
        'ADGROUP': ADGROUP_LIST,
        'URL': URL_LIST,
        'CRITERIA': CRITERIA_LIST,
        'AUDIENCE': BIDDABLE_LIST,
        'AGE_RANGE': BIDDABLE_LIST,
        'DISPLAY_KEYWORD': BIDDABLE_LIST,
    }
class ReportColumn:
    ADGROUP_LIST  = [
        'customer_id', 'campaign_id', 'channel_type', 'adgroup_id', 'status', 'cpm_bid', 'cpv_bid', 'cpc_bid', 'cpa_bid', 'bidding_type', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion']
    URL_LIST      = [
        'customer_id', 'campaign_id', 'adgroup_id', 'status', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion', 'url_display_name']
    CRITERIA_LIST = [
        'customer_id', 'campaign_id', 'adgroup_id', 'keyword_placement', 'keyword_id', 'status', 'cpm_bid', 'cpc_bid', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion']
    AUDIENCE_LIST = [
        'customer_id', 'campaign_id', 'adgroup_id', 'audience', 'criterion_id', 'status', 'cpm_bid', 'cpc_bid', 'bidding_type', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion']
    AGE_RANGE_LIST = [
        'customer_id', 'campaign_id', 'adgroup_id', 'age_range', 'criterion_id', 'status', 'cpm_bid', 'cpc_bid', 'bidding_type', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion']
    BIDDABLE_LIST = [
        'customer_id', 'campaign_id', 'adgroup_id', 'keyword', 'keyword_id', 'status', 'cpm_bid', 'cpc_bid', 'bidding_type', 'spend', 'cost_per_target', 'impressions', 'clicks', 'conversions', 'cost_per_click', 'cost_per_conversion']

    INDEX = {
        'ADGROUP': ADGROUP_LIST,
        'URL': URL_LIST,
        'CRITERIA': CRITERIA_LIST,
        'AUDIENCE': AUDIENCE_LIST,
        'AGE_RANGE': AGE_RANGE_LIST,
        'DISPLAY_KEYWORD': BIDDABLE_LIST,
    }

In [163]:
def get_performance_insights(client, campaign_id=None, adgroup_id=None, date_preset=None, performance_type='ADGROUP'):
    report_downloader = client.GetReportDownloader(version='v201809')
    if date_preset is None:
        date_preset = 'ALL_TIME'
    else:
        date_preset = date_preset
    # Create report definition.
    if campaign_id and not adgroup_id:
        operand = {
            'field': 'CampaignId',
            'operator': 'EQUALS',
            'values': [campaign_id]
        }
#         ReportField.INDEX[performance_type].remove('AdGroupId')
#         ReportColumn.INDEX[performance_type].remove('adgroup_id')
        fields = ReportField.INDEX[performance_type]
        columns = ReportColumn.INDEX[performance_type]
    elif adgroup_id:
        operand = {
            'field': 'AdGroupId',
            'operator': 'EQUALS',
            'values': [adgroup_id]
        }
        fields = ReportField.INDEX[performance_type]
        columns = ReportColumn.INDEX[performance_type]
    else:
        print('get_performance_insights: Missing arguments campaign_id or adgroup_id.')
        operand = None
        return
    report = {
        'downloadFormat': 'CSV',
        'reportName': performance_type+'_PERFORMANCE_REPORT',
        'reportType': performance_type+'_PERFORMANCE_REPORT',
        'dateRangeType': date_preset,
        'selector': {
            'fields': fields,
            'predicates': [operand]
        }
    }
    # Print out the report as a string
    with open(performance_type+'.csv', 'wb') as output_file:
        report_downloader.DownloadReport(
            report, output=output_file, skip_report_header=True, skip_column_header=False,
            skip_report_summary=True, include_zero_impressions=False)
    with open(performance_type+'.csv')as csv_file:
        df = pd.read_csv(csv_file, sep=",", quotechar='"')
        df[df.columns.difference( ReportField.NON_NUMERIC_LIST )] = df[df.columns.difference( ReportField.NON_NUMERIC_LIST )].apply(pd.to_numeric, errors='coerce')
        df[df.columns.intersection( ReportField.NUMERIC_LIST )] = df[df.columns.intersection( ReportField.NUMERIC_LIST )].div(1000000)
        df.columns = columns
        df.sort_values(by=['impressions'], ascending=False).reset_index(drop=True)
        gdn_db.into_table( df, DATABASE_INDEX[performance_type] )
        return df


    

In [164]:
def retrive_all_criteria_insights():
    starttime = datetime.datetime.now()
    df_camp = gdn_db.get_campaign()
    campaign_id_list = df_camp['campaign_id'].unique()
    # retrive all criteria insights
    for campaign_id in campaign_id_list:
        df = df_camp[df_camp.campaign_id==campaign_id]
        customer_id = df['customer_id'].iloc[0]
        destination_type = df['destination_type'].iloc[0]
        client.SetClientCustomerId(customer_id)
        for criteria in CRITERIA_LIST:
            df = get_performance_insights(client, campaign_id=campaign_id, performance_type=criteria)
            

In [152]:
CRITERIA_LIST

['ADGROUP', 'URL', 'CRITERIA', 'AGE_RANGE', 'DISPLAY_KEYWORD', 'AUDIENCE']

In [165]:
%%time
from tabulate import tabulate
df_camp = gdn_db.get_campaign()
campaign_id_list = df_camp['campaign_id'].unique()
for campaign_id in campaign_id_list:
    df = df_camp[df_camp.campaign_id==campaign_id]
    customer_id = df['customer_id'].iloc[0]
    destination_type = df['destination_type'].iloc[0]
    client.SetClientCustomerId(customer_id)
    for criteria in CRITERIA_LIST:
        df = get_performance_insights(client, campaign_id=campaign_id, performance_type=criteria)
#         print(tabulate(df, headers=df.columns, tablefmt='simple'))


[1777436620 1809236789]
1777436620
1777436620
1777436620
1777436620
1777436620
1777436620
1809236789
1809236789
1809236789
1809236789
1809236789
1809236789
CPU times: user 489 ms, sys: 15.9 ms, total: 505 ms
Wall time: 34.8 s


In [6]:
%%time
# import gdn_db
# df_camp = gdn_db.get_campaign()
# campaign_id_list = df_camp['campaign_id'].unique()
# for campaign_id in campaign_id_list:
#     for criteria in CRITERIA_LIST:
# table = DATABASE_INDEX["CRITERIA"]
# df = gdn_db.get_table(campaign_id=1747836664, table=table)
# print(tabulate(df.head(1), headers=df.columns, tablefmt='simple'))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


In [139]:
import gdn_db
BIDDING_INDEX = {
    'cpc': 'cpc_bid',
    'cpa': 'cpa_bid',
}
TARGET_INDEX = {
    'cpc': 'clicks',
    'cpa': 'conversions',
}

def get_criteria_score( campaign_id=None, criteria=None):
    mydb = gdn_db.connectDB("dev_gdn")
    df = pd.DataFrame({'adgroup_id': [], 'target': [],
                       'impressions': [], 'bid_amount': []})
    df_camp = pd.read_sql(
        "SELECT * FROM campaign_target WHERE campaign_id='{}'".format(campaign_id), con=mydb)    
    table = DATABASE_INDEX[criteria]
    df = pd.read_sql(
        "SELECT * FROM {} WHERE campaign_id='{}'".format(table, campaign_id), con=mydb)
    df_weight = pd.read_sql(
        "SELECT * FROM optimal_weight WHERE campaign_id='{}' ".format(campaign_id), con=mydb)
    if not df.empty:
        if criteria != "URL" and criteria != "CRITERIA":
            df['daily_budget'] = df_camp['daily_budget'].iloc[0]
            df['bid_amount'] = df[ BIDDING_INDEX[ df['bidding_type'].iloc[0] ] ]
            df['target'] = df[ TARGET_INDEX[ df['bidding_type'].iloc[0] ] ]
            daily_destination = df_camp['destination'] / df_camp['period']
            df['daily_destination'] = daily_destination.iloc[0]
            for index, row in df.iterrows():
                df = pd.DataFrame(data=[row], columns=df.columns, index=[0])
                r = ObjectiveFunc.adgroup_fitness(df_weight, df)
                df['score'] = r
                df_final = df[ SCORE_COLUMN_INDEX[criteria] ]
                gdn_db.into_table(df_final, table=SCORE_INDEX[criteria]+"_score")
            
            mydb.close()
            return 


In [140]:
get_criteria_score(campaign_id=1777436620, criteria="AGE_RANGE")

AGE_RANGE


In [15]:
from genetic_algorithm_gdn import *
campaign_id = 1777436620
global df
df = ObjectiveFunc().campaign_status(campaign_id)
bound = np.tile([[0], [1]], vardim)
ga = GeneticAlgorithm(sizepop, vardim, bound, MAXGEN, params)
optimal = ga.solve()
score = ObjectiveFunc.fitnessfunc(optimal, df)

score_columns=['weight_kpi', 'weight_spend', 'weight_bid']
df_score = pd.DataFrame(data=[optimal], columns=['weight_kpi', 'weight_spend', 'weight_bid'], index=[0])
#         score_columns=['weight_kpi', 'weight_spend', 'weight_bid', 'weight_width']
#         df_score = pd.DataFrame(data=[optimal], columns=['weight_kpi', 'weight_spend', 'weight_bid', 'weight_width'], index=[0])        

df_final = pd.DataFrame({'campaign_id':campaign_id, 'score':score}, columns=['campaign_id', 'score'], index=[0])
df_final = pd.concat( [df_score, df_final], axis=1, sort=True, ignore_index=False)

print(df_final)
gdn_db.check_optimal_weight(campaign_id, df_final)

/home/tim_su/ai_optimizer/opt/ai_optimizer/ai_optimizer/codes/genetic_algorithm_gdn.py:290: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  df = df.convert_objects(convert_numeric=True)


NameError: name 'df' is not defined

In [161]:
adgroup_id=None
campaign_id=1

In [162]:
if adgroup_id or campaign_id:
    print(True)
elif not adgroup_id:
    print(False)

True
